<a href="https://colab.research.google.com/github/dlrhksghdaf/DS_30/blob/main/4_Pytorch_Multiclass_Classification_(%EC%88%98%EC%97%85%EC%9A%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-class Classification with Softamx

## 1. Softmax Cost function with Pytorch

소프트맥스 함수는 다음과 같이 정의 된다.
$$\sigma(\mathbf{z})_i=\frac{exp({z_i})}{\sum_{j=1}^{K}exp({z_j})}$$

위의 식은 $K$의 개의 classes가 있을 때 $i$ 주어진 $\mathbf{z}$ (보통 각 클래스에 해당하는 scores를 element로 갖는 벡터)를 확률로 바꾸기 위해서 softmax 를 사용한다. (scores -> [0, 1] 범위의 값들로 변환)

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
z = torch.FloatTensor([1, 2, 3])

In [ ]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [ ]:
hypothesis.sum()

tensor(1.)

이제 cost function을 구현

Cross Entropy

일반적으로 Multi-class classification을 할 때는 Cross Entropy를 사용한다.
식은 다음과 같다.
$$\mathcal{L}=-\sum_{j=1}^{K}y_j\log (p_j)$$
여기에서 $K$는 클래스 갯수 이고 $y_j$는 $\mathbf{y}$라는 one-hot vector label의 $j$번째 element를 뜻 하고 $p_j$는 softmax output의 $j$ 번째 element 값을 뜻 한다.

이것을 N개의 전체 데이터에 대해서 확장 시키면 다음과 같다.
$$\mathcal{L}=-\frac{1}{N}\sum_{i=1}^{N} \sum_{j=1}^{K}y_j^{(i)}\log (p_j^{(i)})$$

Binary Classification과의 관계
Logistic Regression에서의 cost function을 다시 써 보면 다음과 같다.
$$\mathcal{L}=-\left(y\log H(x) + (1-y)\log (1-H(x)\right)$$
이것을 cross entropy 관점에서 해석 할 수 있다.

- $y=y_1$
- $1-y=y_2$
- $H(x)=p_1$
- $1-H(x)=p_2$

라고 해석하면 $K=2$인 경우로 볼 수 있다. (Cross Entropy의 Special Case)

$$\mathcal{L}=-\frac{1}{N}\sum_{i=1}^{N}\sum_{j=1}^{2}y_j^{(i)}\log (p_j^{(i)})= -\frac{1}{N}\sum_{i=1}^{N}\left(y^{(i)}\log (p^{(i)})+(1-y^{(i)})\log (1-p^{(i)})\right)$$

In [ ]:
z = torch.rand(3, 5, requires_grad=True)

In [ ]:
z.shape

torch.Size([3, 5])

In [ ]:
F.softmax(z)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.2854, 0.2291, 0.1493, 0.1267, 0.2096],
        [0.2165, 0.2553, 0.2281, 0.1480, 0.1521],
        [0.2942, 0.1997, 0.1503, 0.1384, 0.2174]], grad_fn=<SoftmaxBackward>)

In [ ]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2854, 0.2291, 0.1493, 0.1267, 0.2096],
        [0.2165, 0.2553, 0.2281, 0.1480, 0.1521],
        [0.2942, 0.1997, 0.1503, 0.1384, 0.2174]], grad_fn=<SoftmaxBackward>)


In [ ]:
# Label 생성
y = torch.randint(high=5, size=(3,)).long()
print(y)

tensor([2, 0, 0])


In [ ]:
# One-hot encoding
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(dim=1, index=y.unsqueeze(1), value=1)

tensor([[0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [ ]:
y.unsqueeze(1)

tensor([[2],
        [0],
        [0]])

In [ ]:
print(y_one_hot)

tensor([[0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])


In [ ]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.5520, grad_fn=<MeanBackward0>)


다른 함수 활용하기

1. F.softmax() + torch.log() = F.log_softmax()

In [ ]:
# 처음 구현
torch.log(F.softmax(z, dim=1))

tensor([[-1.2540, -1.4734, -1.9021, -2.0662, -1.5628],
        [-1.5302, -1.3653, -1.4779, -1.9106, -1.8834],
        [-1.2236, -1.6108, -1.8951, -1.9778, -1.5258]], grad_fn=<LogBackward>)

In [ ]:
# F.log_softmax() 활용
torch.log_softmax(z, dim=1)

tensor([[-1.2540, -1.4734, -1.9021, -2.0662, -1.5628],
        [-1.5302, -1.3653, -1.4779, -1.9106, -1.8834],
        [-1.2236, -1.6108, -1.8951, -1.9778, -1.5258]],
       grad_fn=<LogSoftmaxBackward>)

2. F.log_softmax() + F.nll_loss() = F.cross_entropy()

In [ ]:
# 첫 번째 구현
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.5520, grad_fn=<MeanBackward0>)

In [ ]:
# 두 번쨰 구현
(y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.5520, grad_fn=<MeanBackward0>)

In [ ]:
# 세번째 구현
# F.nll_loss()를 사용하면 one-hot vector를 넣을 필요 없음.
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.5520, grad_fn=<NllLossBackward>)

In [ ]:
# 네번째 구현
F.cross_entropy(z, y)

tensor(1.5520, grad_fn=<NllLossBackward>)

## 2. Multi-class Classification Implementation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
# classes: 0, 1, 2
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [ ]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


### 2.1. Low-level Implementation

In [ ]:
# One-hot Encoding
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [ ]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [ ]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### 2.2 High-level Implementation

F.cross_entropy() 사용하기

In [ ]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433506
Epoch 1000/1000 Cost: 0.399962


### 2.3. Implementation with nn.Module

In [ ]:
# 모델을 선언 및 초기화. 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3.
model = nn.Linear(4, 3)

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.560073
Epoch  100/1000 Cost: 0.650332
Epoch  200/1000 Cost: 0.562333
Epoch  300/1000 Cost: 0.507219
Epoch  400/1000 Cost: 0.462687
Epoch  500/1000 Cost: 0.423196
Epoch  600/1000 Cost: 0.386318
Epoch  700/1000 Cost: 0.350517
Epoch  800/1000 Cost: 0.314623
Epoch  900/1000 Cost: 0.278109
Epoch 1000/1000 Cost: 0.246246


### 2.4 Implementatio with class

[연습]

- Data: x_train, y_train

In [ ]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        # Your Code
        super().__init__()
        self.linear = nn.Linear(4, 3)
    
    def forward(self, x):
        # Your Code
        return self.linear(x)

In [ ]:
model = SoftmaxClassifierModel()

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # Your Code
    prediction = model(x_train)

    # cost 계산
    # Your Code
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.249955
Epoch  100/1000 Cost: 0.232704
Epoch  200/1000 Cost: 0.221423
Epoch  300/1000 Cost: 0.211150
Epoch  400/1000 Cost: 0.201753
Epoch  500/1000 Cost: 0.193125
Epoch  600/1000 Cost: 0.185176
Epoch  700/1000 Cost: 0.177831
Epoch  800/1000 Cost: 0.171025
Epoch  900/1000 Cost: 0.164701
Epoch 1000/1000 Cost: 0.158811


## 3. MNIST Classification with Linear Model

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

cuda


In [ ]:
# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# hyperparameters
training_epochs = 15
batch_size = 100

MNIST Classifier

In [ ]:
mnist_train.data.shape

torch.Size([60000, 28, 28])

In [ ]:
mnist_train.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [ ]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# dataset loader
data_loader = DataLoader(dataset=mnist_train,
                         batch_size=batch_size, # 배치 크기는 100
                         shuffle=True,
                         drop_last=True)

In [ ]:
# MNIST data image of shape 28 * 28 = 784
linear = nn.Linear(784, 10, bias=True).to(device)

In [ ]:
# Obejective functio and Optimizer
criterion = nn.CrossEntropyLoss() # 내부적으로 소프트맥스 함수를 포함하고 있음.
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [ ]:
for epoch in range(training_epochs): # 앞서 training_epochs의 값은 15로 지정함.
    avg_cost = 0
    total_batch = len(data_loader)

    for X, Y in data_loader:
        # 배치 크기가 100이므로 아래의 연산에서 X는 (100, 784)의 텐서가 된다.
        X = X.view(-1, 28 * 28).to(device)
        # 레이블은 원-핫 인코딩이 된 상태가 아니라 0 ~ 9의 정수.
        Y = Y.to(device)

        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.534769297
Epoch: 0002 cost = 0.359351158
Epoch: 0003 cost = 0.331293285
Epoch: 0004 cost = 0.316689491
Epoch: 0005 cost = 0.307015419
Epoch: 0006 cost = 0.300365299
Epoch: 0007 cost = 0.295039892
Epoch: 0008 cost = 0.290867805
Epoch: 0009 cost = 0.287298501
Epoch: 0010 cost = 0.284339368
Epoch: 0011 cost = 0.281994671
Epoch: 0012 cost = 0.279755592
Epoch: 0013 cost = 0.277896315
Epoch: 0014 cost = 0.276015371
Epoch: 0015 cost = 0.274435967
Learning finished


Class를 이용해서 코딩 + Custom DataLoader 사용해 보기

In [ ]:
import torch
from torchvision import transforms

# Your Code
class MNISTDataset(torch.utils.data.Dataset):
  def __init__(self, x, y):
    self.x = x.type(torch.float)
    self.y = y
  
  def __len__(self):
    # return length
    return len(self.x)
  
  def __getitem__(self, idx):
    # return x, y
    x = self.x[idx]
    y = self.y[idx]
    return x, y

In [ ]:
x_train = mnist_train.data / 256.
y_train = mnist_train.targets

In [ ]:
# Your Code
mnist_train_data = MNISTDataset(x_train, y_train)
data_loader = DataLoader(dataset=mnist_train_data,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [ ]:
# Your Code
class MultiClassLinear(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(28 * 28, 10)
  
  def forward(self, x):
    return self.linear(x)

In [ ]:
model = MultiClassLinear().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.01)

In [ ]:
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  for x, y in data_loader:
    x = x.view(-1, 28 * 28).to(device)
    y = y.to(device)
    
    hypothesis = model(x)
    cost = criterion(hypothesis, y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
print('Learning finished')

Epoch: 0001 cost = 0.493168205
Epoch: 0002 cost = 0.421722859
Epoch: 0003 cost = 0.413553685
Epoch: 0004 cost = 0.411126137
Epoch: 0005 cost = 0.410578340
Epoch: 0006 cost = 0.410026282
Epoch: 0007 cost = 0.409962118
Epoch: 0008 cost = 0.409748852
Epoch: 0009 cost = 0.409998536
Epoch: 0010 cost = 0.409920335
Epoch: 0011 cost = 0.409634829
Epoch: 0012 cost = 0.409740746


KeyboardInterrupt: ignored